In [17]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [18]:
import sys
from pathlib import Path

# this notebook is in notebooks — define project root
project_root = Path().resolve().parent
sys.path.append(str(project_root))

In [19]:
import gradio as gr
import pandas as pd
from gdc_pipeline import execute_pipeline
from gdc_pipeline import setup_args

In [20]:
# init a dummy list
sys.argv = ['prog', '--question', 'dummy']
args = setup_args()

In [21]:
args

Namespace(input_file=None, question='dummy', intent_model_path='/opt/gpudata/aartiv/qag/query_intent_model.pt', path_to_gdc_genes_mutations_file='/opt/gpudata/aartiv/qag/gdc_genes_mutations.json')

In [26]:
def wrapped_execute_pipeline(question: str):
    df = pd.DataFrame({'questions' : [question]})
    print(f'Question received: {question}')
    try:
        return execute_pipeline(
            df=df,
            intent_model_path=args.intent_model_path,
            path_to_gdc_genes_mutations=args.path_to_gdc_genes_mutations_file,
            output_file_prefix=None
        )
    except Exception as e:
        print('Unable to execute GDC API, can you please retry? {}'.format(str(e)))

In [27]:
question = 'What is the co-occurence frequency of somatic homozygous deletions in CDKN2A and CDKN2B in the mesothelioma project TCGA-MESO in the genomic data commons?'

In [28]:
wrapped_execute_pipeline(question)

Question received: What is the co-occurence frequency of somatic homozygous deletions in CDKN2A and CDKN2B in the mesothelioma project TCGA-MESO in the genomic data commons?
starting pipeline
loading HF token
getting gdc project information
loading gdc genes and mutations
loading llama model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

running test on input                                            questions
0  What is the co-occurence frequency of somatic ...


  0%|          | 0/1 [00:00<?, ?it/s]

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

...

100%|██████████| 1/1 [00:06<00:00,  6.24s/it]


get_prefinal_response took 6.2400 seconds
postprocessing response


100%|██████████| 1/1 [00:00<00:00, 640.45it/s]

                                                                   0
questions          What is the co-occurence frequency of somatic ...
llama_base_output                          The final answer is: 0.0%
helper_output                 joint frequency in TCGA-MESO is 44.83%
cancer_entities                                            TCGA-MESO
gene_entities                                       [CDKN2A, CDKN2B]
mutation_entities                                                 []
modified_prompt    What is the co-occurence frequency of somatic ...
ground_truth_stat                                              44.83
llama_base_stat                                                  0.0
delta_llama                                                   -44.83
final_response                           The final answer is: 44.83%
completed


execute_pipeline took 30.5813 seconds
